In [3]:
from pathlib import Path

import pandas as pd

In [42]:
file = "../runs/retrieved/run.eswiki_20240401.text-embedding-3-large.messirve-v1.0-full-test.tsv.gz"
df_run = pd.read_csv(
    file, sep=" ", header=None, names = ["qid", "Q0", "docid", "rank", "score", "run_name"]
)

In [43]:
df_run.head(2)

,qid,Q0,docid,rank,score,run_name
0,5332643,Q0,23506#50,1,0.572015,FV
1,5332643,Q0,23506#84,2,0.555346,FV


In [41]:
import datasets

ds_qrels = datasets.load_dataset(
    "csv", 
    data_files="https://huggingface.co/datasets/spanish-ir/messirve-trec/resolve/main/qrels.messirve-v1.0-full-test.tsv", 
    split="train", header=None, column_names=["qid", "Q0", "docid", "rel"], sep="\t",
)
ds_queries = datasets.load_dataset(
    "csv", 
    data_files="https://huggingface.co/datasets/spanish-ir/messirve-trec/resolve/main/topics.messirve-v1.0-full-test.tsv", 
    split="train", header=None, column_names=["qid", "query"], sep="\t",
)
ds_docs = datasets.load_dataset("spanish-ir/eswiki_20240401_corpus")


df_qrels = pd.DataFrame(ds_qrels[:])
df_queries = pd.DataFrame(ds_queries[:])
df_docs = pd.DataFrame(ds_docs["corpus"])


In [46]:
# add qrels to run:
df_full = (
    df_run[["qid", "docid", "rank", "score"]]
    .merge(df_qrels[["qid", "docid", "rel"]], on=["qid", "docid"], how="left")
)

In [47]:
df_full["rel"] = df_full["rel"].fillna(0).astype(int)

In [48]:
df_full.head(2)

,qid,docid,rank,score,rel
0,5332643,23506#50,1,0.572015,0
1,5332643,23506#84,2,0.555346,0


In [ ]:
min_hit_ranks = df_full[df_full["rel"] == 1].groupby("qid")["rank"].min()
df_full["min_hit_rank"] = df_full["qid"].map(min_hit_ranks)

In [66]:
df_full["min_hit_rank"].isna().sum()

1719600

In [67]:
import numpy as np

df_full["min_hit_rank"] = df_full["min_hit_rank"].fillna(np.inf)

In [68]:
# Keep cases where relevant doc is not in top 20 results:
df_misses = (
    df_full[df_full["min_hit_rank"] > 20]
    .merge(df_queries[["qid", "query"]], on="qid", how="left")
    .merge(df_docs[["docid", "title", "text"]], on="docid", how="left")
)

In [71]:
df_misses["doc"] = df_misses["title"] + ". " + df_misses["text"]

In [75]:
# Add the actual relevant doc:
df_rel_docs = df_qrels[df_qrels["rel"] == 1].merge(
    df_docs[["docid", "title", "text"]], on="docid", how="left"
)
df_rel_docs["rel_doc"] = df_rel_docs["title"] + ". " + df_rel_docs["text"]
df_rel_docs = df_rel_docs[["qid", "docid", "rel_doc"]].copy()

In [ ]:
df_rel_docs.head(2)
# NOTE there can be some qids with many rel docs!

,qid,docid,rel_doc
0,5332643,87525#2,Efialtes de Atenas. Efialtes es considerado po...
1,5332647,77666#7,Arroba (unidad de masa). En Bolivia y Perú hoj...


In [74]:
(
    df_misses
    .sort_values(["min_hit_rank", "qid", "rank"], ascending=[False, True, True])
    [["query", "min_hit_rank", "doc"]]
)

,query,min_hit_rank,doc
400,18 de octubre que santo se celebra,inf,18 de octubre. El 18 de octubre es el 291.º (d...
401,18 de octubre que santo se celebra,inf,Ricardo Gwyn. Su festividad se celebra el 17 d...
402,18 de octubre que santo se celebra,inf,Marcos (papa). Su festividad es el 7 de octubre.
403,18 de octubre que santo se celebra,inf,San Bartolomé de Bregantia. Se celebra la fest...
404,18 de octubre que santo se celebra,inf,Estacio el Apóstol. Su festividad se celebra e...
...,...,...,...
6574195,uso que se le da al elemento kripton,21.0,Kryptonita (película). Según el sitio web Toda...
6574196,uso que se le da al elemento kripton,21.0,Kryptonita (película). Se tenía previsto lanza...
6574197,uso que se le da al elemento kripton,21.0,Anexo: Isótopos de kriptón. Kriptón-85 es un r...
6574198,uso que se le da al elemento kripton,21.0,"Krypto. Finalmente, Krypto en varias formas fu..."


In [77]:
# Sample 30 qids:
qids_sample = df_misses["qid"].drop_duplicates().sample(30, random_state=36)
df_sample = df_misses[df_misses["qid"].isin(qids_sample)].copy()

In [97]:
# For each qid, show the top 5 returned results and the actual relevant docs:
import textwrap


for qid in df_sample["qid"].unique():
    df_tmp = df_sample[df_sample["qid"] == qid].copy()
    query = df_tmp['query'].values[0]
    top_docs = df_tmp["doc"].head(5)
    rel_docs = df_rel_docs[df_rel_docs["qid"] == qid]["rel_doc"]
    print(f"Query {qid}: {query}")
    print("\n\tTop 5 ranked docs:")
    for i, doc in enumerate(top_docs, 1):
        # print(textwrap.fill(f"{i} - {doc}", 180))
        print(f"\t{i} - {doc}")
    print("\n\tRelevant docs:")
    for i, doc in enumerate(rel_docs, 1):
        print(f"\t{i} - {doc}", 180)
    print()
    print("-"*80)


Query 5420913: cual es el ave de rapiña mas grande del mundo

	Top 5 ranked docs:
	1 - Vultur gryphus. Es el ave voladora más grande del mundo por la medida combinada de peso ( kg) y envergadura máxima ( m). Generalmente se le considera el ave rapaz más grande del mundo. Tiene plumaje negro con un collar de plumas blancas que rodea la base del cuello y, especialmente en el macho, grandes manchas blancas en las alas. La cabeza y el cuello casi no tienen plumas y son de un color rojo apagado, que puede enrojecerse y, por lo tanto, cambiar de color en respuesta al estado emocional del ave. El macho tiene una barba en el cuello y una gran cresta o carúncula de color rojo oscuro en la coronilla. La hembra es más pequeña que el macho, una excepción a la regla entre las aves de presa.
	2 - Águila. Entre las águilas se encuentran algunas de las mayores aves rapaces: sólo el cóndors y algunos de los buitres del Viejo Mundos son notablemente mayores. Se discute regularmente cuál debe considerars

In [ ]:
# error analysis annotation: 

# Query 5420913: cual es el ave de rapiña mas grande del mundo
# -- annotation error (mithological creature)
# Query 5420996: cual es el banco itau
# -- ambiguous query
# Query 5494087: cuanto debes medir para ser militar
# -- false negatives (many militaries mentioned)
# Query 5516819: cuantos japoneses hay en el mundo
# -- false negatives
# Query 5525668: cuantos puntos máximo se pueden tener
# -- ambiguous query
# Query 5531200: cuál es el kalanchoe comestible
# -- right answer not clear
# Query 5534521: cuál fue el gol más rápido de la historia
# -- ambiguous query
# Query 5590269: de donde salio el fascismo
# -- false negatives
# Query 5631525: donde grabaron downton abbey
# -- false negatives
# Query 5641975: donde se encuentra el calamar mas grande
# -- annotation error (not the largest)
# Query 5654608: dónde queda río frío
# -- ambiguous query (many places called rio frio)
# Query 5667923: en donde venden flores artificiales
# -- false negatives
# Query 5678309: en que consisten las encomiendas
# -- false negatives
# Query 5694828: en que viajaban los vikingos
# -- false negatives
# Query 5696114: en qué canal dan el día menos pensado
# -- false negatives
# Query 5704705: en qué zona climática del planeta se encuentra ubicada colombia
# -- false negatives
# Query 5719184: nombre de quimioterapia
# -- ambiguous query
# Query 5836842: que animal no muere
# -- false negatives
# Query 5839343: que autos entran a decreto
# -- ambiguous query
# Query 5863952: que es ondas de choque en fisioterapia
# -- false negatives
# Query 5900694: que pasa si tomo ketorolaco estando embarazada
# -- false negatives
# Query 5907264: que puedo echarle a mi perro para las pulgas
# -- false negatives (but ground truth seems better!!!)
# Query 5916905: que significa mesa en griego
# -- annotation error + model error (they answer about mesa in spanish)
# Query 5954152: quien lidero el fascismo
# -- false negatives
# Query 5964681: quienes formaron la junta provisional de gobierno
# -- ambiguous query
# Query 5966866: quienes pelearon en la guerra de los 100 años
# -- false negatives
# Query 5975134: quién fue el nigromante
# -- false negatives
# Query 5991313: qué era geológica se caracteriza por la aparición de los primeros mamíferos y aves
# -- false negatives
# Query 5997857: qué hizo manuela espejo por las mujeres
# -- annotation error
# Query 6011587: qué significa karina
# -- model error